[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-2-vector-search-openai/vector-search-openai.ipynb)

# Lab: Vector Search on Mongo Atlas Using OpenAI Embeddings


This is a companion notebook for this [TODO - Quick start guide](#)
It will demonstrate the following:

- 👉 Creating a vector index on Atlas
- 👉 Performing vector search using OpenAI embeddings


### What you need to run this notebook

- a (free) MongoDB Atlas Account
- An Atlas instance running in the cloud with sample data loaded
- and connection credentials
- OpenAI API key (optional, see below)

Follow this [TODO quick start guide](#) to set this up before proceeding.

### How to run

This notebook can be run on Google Colab and stand alone python development environments.  Click here to run on colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-2-vector-search-openai/vector-search-openai.ipynb)


References

- https://cookbook.openai.com/examples/vector_databases/mongodb_atlas/semantic_search_using_mongodb_atlas_vector_search

## Step-1: Setup Atlas

We will need to have Atlas setup.

Follow [instructions here](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/lab-1-atlas-setup/setup-atlas.md)

Also the [TODO quick start guide](#) has more information.

## Step-2: Create an Alas Index

Before we can run vector search we need to create a vector index.  Here is how.

### 2.1 - Navigate to Atlas Vector Search UI

<!-- ![atlas-search-2.png](../images//atlas-index-1.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-1.png)

### 2.2 - Choose `Create Vector Index`

<!-- ![atlas-search-2.png](../images//atlas-index-2.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-2.png)

### 2.3 - Create a vector index as follows

Index name: `idx_plot_embedding`

Index definition

```json
{
  "fields": [
    {
      "type": "vector",
      "path": "plot_embedding",
      "numDimensions": 1536,
      "similarity": "dotProduct"
    }
  ]
}
```

<!-- ![atlas-search-3.png](../images//atlas-index-3b.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-3b.png)


### 2.4 - Wait till the index is ready to be used

<!-- ![atlas-search-4.png](../images//atlas-index-4.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-index-4.png)


## Step-3: Configuration

We need to configure the following
- Atlas connection credentials
- openAI API key

### Option 3A - If running on Colab

- Click on 'Colab secrets' icon (🔑) on left pane, and crate the following secrets.
- `ATLAS_URI` and `OPENAI_API_KEY`
-  Make sure the `notebook access` button is checked on for both
- See screenshot below for example

<!-- ![](../images/colab-secret-2.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/colab-secret-2.png)


### Option 3B - If running on local python environment

- setup your local python env following this [setup guide](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/setup-python-env.md)
- Create a file named `.env` in the same location as notebook
- And add the following settings

```text
ATLAS_URI=mongodb+srv://<username>:<password>@sandbox.....
OPENAI_API_KEY=sk...
```


In [1]:
# We will keep all global variables in an object to not pollute the global namespace.
class MyConfig(object):
    pass

MY_CONFIG = MyConfig()

### Determine Runtime Environment

This code will figure out if we are running on Google Colab environment or local environment.  We use it to install relevant packages later.

In [2]:

# are we running in Colab?
import os

if os.getenv("COLAB_RELEASE_TAG"):
    print("Running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = True
else:
    print("NOT running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = False

Running in Colab


## Step-4: Install dependencies (if necessary)

We will install required libraries in cloud environments like Google Colab.  For local environments, we assume the dependencies are already setup.

In [3]:
if MY_CONFIG.RUNNING_IN_COLAB:
    !pip install \
                openai==1.13.3 \
                pymongo==4.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


## Step-5 - Load Configurations

In [4]:
## Load settings based on where we are running
##  - if runninning on google Colab, load from secrets
##  - if running locally use dotenv

if MY_CONFIG.RUNNING_IN_COLAB:
    from google.colab import userdata
    MY_CONFIG.ATLAS_URI = userdata.get('ATLAS_URI')
    MY_CONFIG.OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
else:
    import os, sys
    from dotenv import find_dotenv, dotenv_values

    this_dir = os.path.abspath('')
    parent_dir = os.path.dirname(this_dir)
    sys.path.append (os.path.abspath (parent_dir))

    config = dotenv_values(find_dotenv())
    # debug
    # print (config)
    MY_CONFIG.ATLAS_URI = config.get('ATLAS_URI')
    MY_CONFIG.OPENAI_API_KEY = config.get("OPENAI_API_KEY")
## --- end load config

## If you just want to quickly set the config manually, you can do so here.
# MY_CONFIG.ATLAS_URI = ''
# MY_CONFIG.OPENAI_API_KEY = ''

if not MY_CONFIG.ATLAS_URI:
    raise Exception ("'ATLAS_URI' is not set.  Please set it above to continue...")

# if not MY_CONFIG.OPENAI_API_KEY:
#     raise Exception ("'OPENAI_API_KEY' is not set.  Please set it above to continue...")

## Step-6: AtlasClient and OpenAIClient

Here are couple of handy classes.

For full implementation see here:

- [AtlasClient.py](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/AtlasClient.py) - a handy class to interact with Atlas
- [OpenAIClient.py](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/OpenAIClient.py) - a handy class to intereact with openAI

In [5]:
from pymongo import MongoClient

class AtlasClient ():

    def __init__ (self, altas_uri, dbname):
        self.mongodb_client = MongoClient(altas_uri)
        self.database = self.mongodb_client[dbname]

    ## A quick way to test if we can connect to Atlas instance
    def ping (self):
        self.mongodb_client.admin.command('ping')

    def get_collection (self, collection_name):
        collection = self.database[collection_name]
        return collection

    def find (self, collection_name, filter = {}, limit=10):
        collection = self.database[collection_name]
        items = list(collection.find(filter=filter, limit=limit))
        return items

    # https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/
    def vector_search(self, collection_name, index_name, attr_name, embedding_vector, limit=5):
        collection = self.database[collection_name]
        results = collection.aggregate([
            {
                '$vectorSearch': {
                    "index": index_name,
                    "path": attr_name,
                    "queryVector": embedding_vector,
                    "numCandidates": 50,
                    "limit": limit,
                }
            },
            ## We are extracting 'vectorSearchScore' here
            ## columns with 1 are included, columns with 0 are excluded
            {
                "$project": {
                    '_id' : 1,
                    'title' : 1,
                    'plot' : 1,
                    'year' : 1,
                    "search_score": { "$meta": "vectorSearchScore" }
            }
            }
            ])
        return list(results)

    def close_connection(self):
        self.mongodb_client.close()


In [6]:
from openai import OpenAI

class OpenAIClient():
    def __init__(self, api_key) -> None:
        self.client = OpenAI(
            api_key= api_key,  # defaults to os.environ.get("OPENAI_API_KEY")
        )
        # print ("OpenAI Client initialized!")


    def chat (self, messages, model="gpt-3.5-turbo"):
        chat_completion = self.client.chat.completions.create(
                        messages=messages, model=model,)
        return chat_completion

    def get_embedding(self, text: str,  model="text-embedding-ada-002") -> list[float]:
        text = text.replace("\n", " ")
        resp = self.client.embeddings.create (
            input=[text],
            model=model  )

        return resp.data[0].embedding

## Step-7: Initialize Mongo Atlas Client

See if we can connect to our Atlas cloud instance.

If this step fails, make sure 'connect from anywhere' is enabled on your Atlas network configuration

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-connect-2.png)

In [7]:
MY_CONFIG.DB_NAME = 'sample_mflix'
MY_CONFIG.COLLECTION_NAME = 'embedded_movies'
MY_CONFIG.INDEX_NAME = 'idx_plot_embedding'

In [8]:
atlas_client = AtlasClient (MY_CONFIG.ATLAS_URI, MY_CONFIG.DB_NAME)
atlas_client.ping()
print ('Connected to Atlas instance! We are good to go!')

Connected to Atlas instance! We are good to go!


## Step-8: Initialize OpenAI Client

In [9]:
openAI_client = None

if MY_CONFIG.OPENAI_API_KEY:
    openAI_client = OpenAIClient (api_key=MY_CONFIG.OPENAI_API_KEY)
    print ("OpenAI client initialized")

OpenAI client initialized


## Step-9: Cached Embeddings

To make you get up and running quickly, we have cached some embedding results.  This way we can query Atlas without having to call embedding API first.

If you use these sample queries, you won't need an OpenAI Key.  If you want to try a different query, then you will need an openAI API key.

In [10]:
## Download the embeddings
cached_embedding_file = 'embeddings_openai.json'

if MY_CONFIG.RUNNING_IN_COLAB and not os.path.exists(cached_embedding_file):
    !wget  -O  {cached_embedding_file}  'https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/lab-2-vector-search-openai/embeddings_openai.json'

--2024-03-10 10:01:10--  https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/lab-2-vector-search-openai/embeddings_openai.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241301 (236K) [text/plain]
Saving to: ‘embeddings_openai.json’

embeddings_openai.j 100%[===================>] 235.65K  --.-KB/s    in 0.02s   

2024-03-10 10:01:10 (14.1 MB/s) - ‘embeddings_openai.json’ saved [241301/241301]



In [11]:
import os
import json

cached_embeddings = {}

if os.path.exists(cached_embedding_file):
    with open(cached_embedding_file, "r") as f:
        str = f.read()
        cached_embeddings = json.loads(str)

print ("Loaded the following cached embeddings...")
for query in cached_embeddings.keys():
    print (f'- {query}')

Loaded the following cached embeddings...
- fatalistic sci-fi movies
- humans fighting aliens
- futuristic christmas movies
- sci-fi story with a friendly alien
- relationship drama between two good friends
- college graduates working in a big city discover new relationships
- household pets get lost but go on a long journey to find home


## Step-10: Do a Vector Search

Now that we have every thing setup, this is the fun part!

We are going to query movies, not just on plot keywords but 'meaning'.

See the examples below.  And try your own!

The process is as follows:

- convert query into embeddings (using OpenAI API)
- send the embeddings to Atlas and get results

### Note the Score

IN addition to movie attributes (title, year, plot ..etc) We are also dislaying `search_score`.  This is a meta attribute - not really part of movies collection, but generated as a result of vector search.

This is a number between 0 and 1.  Closer to 1 values represent 'better match'.  And the results are sorted from best match down (closer to 1 first)

[You can read more about search score here](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#atlas-vector-search-score)


### Troubleshooting

#### No search results?

Make sure the vector search index is defined and active! (Step-2)

In [12]:
import time

# Handy function
def do_vector_search (query:str) -> None:
    query = query.lower().strip()
    print ('query: ', query)
    if query in cached_embeddings.keys():
        print ("using cached embeddings")
        embedding = cached_embeddings.get (query)
    else:
        t1a = time.perf_counter()
        embedding = openAI_client.get_embedding(query)
        t1b = time.perf_counter()
        print (f"Getting embeddings from OpenAI took {(t1b-t1a)*1000:,.0f} ms")

    t2a = time.perf_counter()
    movies = atlas_client.vector_search(collection_name=MY_CONFIG.COLLECTION_NAME, index_name=MY_CONFIG.INDEX_NAME, attr_name='plot_embedding', embedding_vector=embedding,limit=10 )
    t2b = time.perf_counter()

    print (f"Altas query returned {len (movies)} movies in {(t2b-t2a)*1000:,.0f} ms")
    print()

    for idx, movie in enumerate (movies):
        print(f'{idx+1}\nid: {movie["_id"]}\ntitle: {movie["title"]},\nyear: {movie["year"]}' +
            f'\nsearch_score(meta):{movie["search_score"]}\nplot: {movie["plot"]}\n')

In [13]:
query="humans fighting aliens"

do_vector_search (query=query)

query:  humans fighting aliens
using cached embeddings
Altas query returned 10 movies in 138 ms

1
id: 573a1398f29313caabce8f83
title: V: The Final Battle,
year: 1984
search_score(meta):0.9573556184768677
plot: A small group of human resistance fighters fight a desperate guerilla war against the genocidal extra-terrestrials who dominate Earth.

2
id: 573a13c7f29313caabd75324
title: Falling Skies,
year: 2011è
search_score(meta):0.9550596475601196
plot: Survivors of an alien attack on earth gather together to fight for their lives and fight back.

3
id: 573a139af29313caabcf0cff
title: Starship Troopers,
year: 1997
search_score(meta):0.9523435831069946
plot: Humans in a fascistic, militaristic future do battle with giant alien bugs in a fight for survival.

4
id: 573a139ff29313caabd000f6
title: Battlefield Earth,
year: 2000
search_score(meta):0.9512618780136108
plot: After enslavement & near extermination by an alien race in the year 3000, humanity begins to fight back.

5
id: 573a139af29

In [14]:
query="relationship drama between two good friends"

do_vector_search (query=query)

query:  relationship drama between two good friends
using cached embeddings
Altas query returned 10 movies in 71 ms

1
id: 573a13a3f29313caabd0dfe2
title: Dark Blue World,
year: 2001
search_score(meta):0.9380425214767456
plot: The friendship of two men becomes tested when they both fall for the same woman.

2
id: 573a13a3f29313caabd0e14b
title: Dark Blue World,
year: 2001
search_score(meta):0.9380425214767456
plot: The friendship of two men becomes tested when they both fall for the same woman.

3
id: 573a1399f29313caabcec488
title: Once a Thief,
year: 1991
search_score(meta):0.9260045289993286
plot: A romantic and action packed story of three best friends, a group of high end art thieves, who come into trouble when a love-triangle forms between them.

4
id: 573a13b3f29313caabd3b197
title: Hulchul,
year: 2004
search_score(meta):0.9249595403671265
plot: A man and woman from feuding families each pretend to fall in love, as part of a revenge plot. Chaos ensues when their fake romance bec

### Try your own searches!

Update the query string to what ever you like, and run it.

Remember, if you want to try different queries, than what we cached, you will need your OPENAI_API_KEY

In [15]:
## TODO: enter your query here
# query="technology gone wrong"

# do_vector_search (query=query)


In [16]:
## Close connection

# atlas_client.close_connection()